# Init Spark and build Spark session

In [1]:
import numpy as np
import pandas as pd
import findspark
findspark.init('/home/ywx-data/spark/spark-2.4.3-bin-hadoop2.7')

import pyspark
sc = pyspark.SparkContext(appName=None)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(None).getOrCreate()

In [3]:
spark

# Read and show raw data with Spark DataFrame

In [4]:
df_raw = spark.read.csv("lh_tp_node_ui.csv", header=True, inferSchema=True)
input_lh_tp_node_ui = df_raw.filter('lv == 1')

In [7]:
df_raw.printSchema()

root
 |-- LV: integer (nullable = true)
 |-- NODE: long (nullable = true)
 |-- L1: string (nullable = true)
 |-- DATA_TIME: timestamp (nullable = true)
 |-- U: double (nullable = true)
 |-- IR: double (nullable = true)
 |-- IX: double (nullable = true)



In [6]:
df_raw.show()

+---+------------+---+-------------------+-----------+------------+------------+
| LV|        NODE| L1|          DATA_TIME|          U|          IR|          IX|
+---+------------+---+-------------------+-----------+------------+------------+
|  1|354010389381|  A|2019-04-18 08:00:00|229.1714286| 7.873935201| 20.07064388|
|  1|354010389388|  A|2019-04-20 20:00:00|    225.875| 8.138034736| 6.975577399|
|  1|354010389381|  A|2019-04-18 08:30:00|229.9428571| -0.98901879|-9.586840019|
|  1|354010389386|  A|2019-04-19 12:30:00|     228.24|-0.100734942| 0.968079481|
|  1|354019022841|  A|2019-04-20 13:15:00|      228.0| 1.269984777| 4.580591726|
|  1|354013332709|  A|2019-04-18 01:30:00|231.7666667|  12.4127701|-2.413129017|
|  1|354010389386|  A|2019-04-18 13:45:00|     229.26|-7.112555611| 12.62412582|
|  1|354010389388|  A|2019-04-20 09:45:00|     227.55| 16.40039602| 1.763531565|
|  1|354010389383|  A|2019-04-20 13:45:00|228.5428571| 19.45274478|-8.603976663|
|  1|354013332709|  A|2019-0

In [8]:
cnt = input_lh_tp_node_ui.select("node").distinct().count()
cnt

12

In [11]:
df = pd.DataFrame(np.random.random((1, 7)))
secondary_df = spark.createDataFrame(df, schema=['node', 'phase', 'rsquare', 'r', 'x', 'b0', 'b1'])
primary_list = []
secondary_df

DataFrame[node: double, phase: double, rsquare: double, r: double, x: double, b0: double, b1: double]

In [12]:
df

,0,1,2,3,4,5,6
0,0.24349,0.017483,0.606333,0.455034,0.494933,0.152829,0.272122


In [13]:
secondary_df.show()

+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+
|               node|              phase|           rsquare|                 r|                 x|                b0|                b1|
+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+
|0.24348994365560517|0.01748258344313236|0.6063326807384616|0.4550337208839601|0.4949326310635016|0.1528285991805417|0.2721223319367121|
+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+



In [14]:
input_lh_tp_node_ui.show()

+---+------------+---+-------------------+-----------+------------+------------+
| LV|        NODE| L1|          DATA_TIME|          U|          IR|          IX|
+---+------------+---+-------------------+-----------+------------+------------+
|  1|354010389381|  A|2019-04-18 08:00:00|229.1714286| 7.873935201| 20.07064388|
|  1|354010389388|  A|2019-04-20 20:00:00|    225.875| 8.138034736| 6.975577399|
|  1|354010389381|  A|2019-04-18 08:30:00|229.9428571| -0.98901879|-9.586840019|
|  1|354010389386|  A|2019-04-19 12:30:00|     228.24|-0.100734942| 0.968079481|
|  1|354019022841|  A|2019-04-20 13:15:00|      228.0| 1.269984777| 4.580591726|
|  1|354013332709|  A|2019-04-18 01:30:00|231.7666667|  12.4127701|-2.413129017|
|  1|354010389386|  A|2019-04-18 13:45:00|     229.26|-7.112555611| 12.62412582|
|  1|354010389388|  A|2019-04-20 09:45:00|     227.55| 16.40039602| 1.763531565|
|  1|354010389383|  A|2019-04-20 13:45:00|228.5428571| 19.45274478|-8.603976663|
|  1|354013332709|  A|2019-0

In [15]:
def get_corr_max_two_node_name(input_lh_tp_node_ui):
    '''
    得到电压相关性最大的两个节点的名称
    :param data: 原始数据
    :return: 节点名 （电表名称）
    '''
    # 生成两两节点组合的名称集合 node_couple
    node_array = np.array(input_lh_tp_node_ui.select("node").distinct().collect()).tolist()
    node_couple = []
    for i in range(len(node_array)):
        for j in range(i + 1, len(node_array)):
            for p in ['A', 'B', 'C']:
                if i != j:
                    node_couple.append([node_array[i][0], node_array[j][0], p])
    print('node_num: %s' %len(node_couple))
    # 利用电压相关系数找出相关性最大的两个点
    u_r2 = []
    for no1, no2, phase in node_couple:
        s1 = "node == %s" % no1
        node1 = input_lh_tp_node_ui.filter(s1)
        node1 = node1.select('data_time', 'u', 'l1').withColumnRenamed('u', 'u1')
        s2 = "node == %s" % no2
        node2 = input_lh_tp_node_ui.filter(s2)
        node2 = node2.select('data_time', 'u', 'l1').withColumnRenamed('u', 'u2').withColumnRenamed('l1', 'l2')
        node_join = node1.join(node2, (node1.data_time == node2.data_time) & (node1.l1 == node2.l2))
        node_join = node_join.coalesce(10)
        u_rsquare = node_join.corr('u1', 'u2')
        u_r2.append([no1, no2, phase, u_rsquare])
    name_u_r2 = ["no1", "no2", "phase", "u_rsquare"]
    u_r2 = pd.DataFrame(columns=name_u_r2, data=u_r2)
    node_x, node_y, phase_v, u_rsquare_best = u_r2[u_r2['u_rsquare'] == u_r2['u_rsquare'].max()].iloc[0,]
    print("In this circulation, node {} and node {} is best!".format(node_x, node_y))
    return node_x, node_y

In [16]:
q = 0
print("{} correlation data num of partition : {}".format(q, input_lh_tp_node_ui.rdd.getNumPartitions()))
node_x, node_y = get_corr_max_two_node_name(input_lh_tp_node_ui)

0 correlation data num of partition : 1
node_num: 198
In this circulation, node 354010389384 and node 354019022841 is best!


In [19]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
def linear_regression(node_x, node_y, node_join_xy, phase='A'):
    '''
    两节点在单个相位上做线性回归分析
    :param node_x: node_x name
    :param node_y: node_y name
    :node_join_xy: node_x node_y拼接后的数据
    :param phase: 相位
    :return: 回归系数列表
    '''
    # 筛选出单相位的数据
    s3 = 'l1 == "%s"' % phase
    node_join_p = node_join_xy.filter(s3)
    node_join_p = node_join_p.drop('node1', 'node2', 'l1')
    assembler = VectorAssembler(inputCols=["u1", "ir1", "ix1", "ir2", "ix2"], outputCol="features")
    output = assembler.transform(node_join_p)
    label_features = output.select("features", "u2").toDF('features', 'label')
    lr = LinearRegression(maxIter=5, elasticNetParam=0.8)
    lrModel = lr.fit(label_features)
    trainingSummary = lrModel.summary
    param = [node_x, node_y, phase, trainingSummary.r2,
             lrModel.intercept,
             lrModel.coefficients[0],
             lrModel.coefficients[1],
             lrModel.coefficients[2],
             lrModel.coefficients[3],
             lrModel.coefficients[4]]
    return param

# from pyspark.ml.regression import LinearRegression
def get_linear_regression_param_list(data, node_x, node_y):
    '''
    两节点在A、B、C三个相位上分别做线性回归
    :param data: 原始数据
    :param node_x: node_x name
    :param node_y: node_y name
    :return: 不同相位的回归系数 ~ spark-df/ node_x,node_y合并后的数据
    '''
    # 生成做回归分析的数据
    s_x = "node == {}".format(node_x)
    s_y = "node == {}".format(node_y)
    nodex = data.filter(s_x)
    nodey = data.filter(s_y)
    nodex = nodex.withColumnRenamed('node', 'node1').withColumnRenamed('u', 'u1').withColumnRenamed('ir',
                                                                                                    'ir1').withColumnRenamed(
        'ix', 'ix1')
    nodey = nodey.withColumnRenamed('node', 'node2').withColumnRenamed('l1', 'l2').withColumnRenamed('u',
                                                                                                     'u2').withColumnRenamed(
        'ir', 'ir2').withColumnRenamed('ix', 'ix2').withColumnRenamed('data_time', 'data_time2')
    node_join_xy = nodex.join(nodey, ((nodex['data_time'] == nodey.data_time2) & (nodex['l1'] == nodey.l2)))
    node_join_xy = node_join_xy.select('node1', 'node2', 'data_time', 'l1', 'u1', 'ir1', 'ix1', 'u2', 'ir2', 'ix2')
    node_join_xy = node_join_xy.withColumn("ir2", node_join_xy["ir2"] * (-1))
    node_join_xy = node_join_xy.withColumn("ix2", node_join_xy["ix2"] * (-1))
    node_join_xy = node_join_xy.coalesce(10)
    # 获得两表~三相位~的回归系数列表
    param_list = []
    for phase in ['A', 'B', 'C']:
        param_list.append(linear_regression(node_x, node_y, node_join_xy, phase=phase))
    name = ['node1', 'node2', 'phase', 'rsquare', 'b0', 'b1', 'r1', 'x1', 'r2', 'x2']
    param_df = pd.DataFrame(columns=name, data=param_list)
    param_dfs = spark.createDataFrame(param_df)
    return param_dfs, node_join_xy

In [20]:
# 得到两个点线性回归后的回归系数 - dfs
# 19/07/14 03:34:09 WARN Column: Constructing trivially true equals predicate, 'data_time#13 = data_time#13'. Perhaps you need to use aliases.
print("{} regression data num of partition : {}".format(q, input_lh_tp_node_ui.rdd.getNumPartitions()))
param_dfs, node_join_xy = get_linear_regression_param_list(input_lh_tp_node_ui, node_x, node_y)
node_join_xy = node_join_xy.coalesce(10)

0 regression data num of partition : 1


In [21]:
param_dfs.show()
node_join_xy.show()

+------------+------------+-----+------------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+
|       node1|       node2|phase|           rsquare|                 b0|                b1|                  r1|                  x1|                  r2|                  x2|
+------------+------------+-----+------------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+
|354010389384|354019022841|    A|0.9959714867818102|0.14585444688059981|0.9993639686110559|0.001373987375185...|-0.09034395738334752|0.003069090407932825|-0.00244922333282...|
|354010389384|354019022841|    B|0.9969139353209018| 1.7413196537090092|0.9922098666653818|-0.20479796583395626| 0.18314146966817496|-4.75272799809889E-4|0.004735318913674037|
|354010389384|354019022841|    C|0.9965693684849772| 0.5610706920182464|0.9973047817358628|    0.13395822549546|0.042889

In [22]:
# 更新输入数据：从原始数据中删除两个子节点，添加新的父节点
input_lh_tp_node_ui = updata_node_couple(input_lh_tp_node_ui, node_join_xy, param_dfs, node_x, node_y, q)
print("before modify,input_lh_tp_node_ui num of partition : {}".format(input_lh_tp_node_ui.rdd.getNumPartitions()))
input_lh_tp_node_ui = input_lh_tp_node_ui.coalesce(10)
print("after modify,input_lh_tp_node_ui's partition: {}".format(input_lh_tp_node_ui.rdd.getNumPartitions()))
print('%s loop new_data lines_num:%s' % (q, input_lh_tp_node_ui.count()))
print('other running time: %s Seconds' % (time.time() - a))

# 主副表单条数据生成、添加
s1_dfs, s2_dfs, p1_list, p2_list = get_primary_secondary_single_data(param_dfs, node_x, node_y, q)
primary_list.append(p1_list)
primary_list.append(p2_list)
secondary_df = secondary_df.union(s1_dfs)
secondary_df = secondary_df.union(s2_dfs)
secondary_df = secondary_df.coalesce(10)
print('%s all running time: %s' % (q, time.time() - a))

NameError: name 'updata_node_couple' is not defined